# Genomic Prediction - 実習編 -

本実習では、ゲノミック予測を実際のイネNAM集団のデータを用いて体験します。

ゲノミック予測モデルを用いることで、どの様なことが出来るのかを実習します。

## 本ノートブックの内容

* 実データを用いてゲノミック予測モデルを構築する

  * 例としてイネのNAM集団、籾数を形質として扱う。

* ゲノミック予測モデルの活用
  * 籾数を増加させる遺伝子型を探索する。
  * 籾数を増加させるために最適な交配系統の組み合わせを探索する。

# はじめに

## Google Colaboratoryの使い方

本実習では、**Google Colaboratory**（以下、**Google Colab**）と呼ばれるオンラインサービスを使って、ゲノミック予測の体験をしていきます。

Google ColabはGoogle社が提供しているサービスで、Googleアカウントさえあれば、オンライン上でPythonやRのプログラミングをおこなえます。無料でそこそこの計算資源を使えるため、実際の研究現場でも利用されています。

細かい使い方は今回は扱いませんが、興味のある方は調べてみてください。

### コードの動かし方

Google Colabには**コードセル**と呼ばれるプログラムを書くスペースがあります。コードセルを選択すると左に表示される「▶︎マーク」をクリックすると、そのコードセルに書かれたプログラムを実行できます。

下のコードのx,y,zの値をいろいろ書き換えて動かしてみてください。

In [ ]:
## --- 入力値 ---
x = 11  # 変数x
y = 22  # 変数y
z = 33  # 変数z
##------------------

## 数値計算
print(x - y + z)

本実習ではこちらで用意したプログラムを動かしてもらう形になるので、プログラミング等をする必要はありません。コードの動かし方が分かればOKです。

それでは実習を始めていきます。

## Mut-Map, QTL-seq, GWASなどの解析

昨日の講義で、形質に関連している遺伝的な変異を検出するQTL-seqを扱いました。

GWASやQTL-seq、Mut-mapなどの手法は**形質に関連しているQTLや遺伝的な変異の検出**にフォーカスした手法になります。

そのため、これらの手法を適用し、有用な遺伝的変異を見つけることで、**Marker assisted selection (MAS)** が行えるようになります。

様々な作物の収量や病害抵抗性などの様々な形質がマーカー選抜を通じて向上しています。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_1.png?raw=true" alt="title" height="150px">

しかし、多くの農業形質は単独の遺伝子(マーカー)によってコントロールされているわけではなく、多数の遺伝子によってコントロールされていることが多いです。

また、集団の形質分離状況などに基づくMut-mapやQTL-seq、GWAS等の手法では、多数の遺伝子が関与していた場合、それぞれの遺伝子の効果を推定することが難しい場合が多いです。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_2.png?raw=true" alt="title" height="300px">

もし、各遺伝的変異の影響力を正しく推定出来た場合、より効率的に育種へマーカーが利用できます。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_3.png?raw=true" alt="title" height="140px">

**ゲノミック予測モデル**は、遺伝子型から形質情報を予測するモデルであり、モデル構築の仕方によっては、各遺伝的変異の形質に対する影響力を推定することが出来る手法の１つになります。

本実習では、ゲノミック予測モデルの基礎をおさらいしつつ、構築方法や予測モデルを用いてどの様なことが出来るのかを実習してもらいます。

# ゲノミック予測モデルについて

ゲノミック予測モデルは形質値を遺伝子型情報から予測モデルになります。

```
通常ゲノミック予測では"育種価"を予測するモデルを指しますが、
育種価という概念は作物育種においてはそれほど重要でないと考えているので、
直接形質値を予測しに行きます。
```

通常、ここでの遺伝子型情報とはSNPの遺伝子型を扱う場合が多いです。

そのため、本実習で扱うゲノミック予測モデルは、SNPの遺伝子型情報を基に形質値を予測するモデルを指します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/genomic_prediction.png?raw=true" alt="colab" height="300px">

ゲノミック予測モデルを構築するためには、遺伝子型と形質値の関係性を捉える必要があります。

そのため、モデルの構築には様々な遺伝子型と対応する形質データが必要です。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_19.png?raw=true" alt="title" height="250px">

大規模な遺伝子型と形質データを用意し、統計モデルを適用することで、遺伝子型と形質データの関係性を説明可能な統計モデルを構築します。

モデルの構築には、基本的には線形モデルが良く用いられ、必要に応じて(深層学習等で用いられる)複雑な統計モデルを使用する場合もありますが、よほど大規模なデータを準備できなければ、効果は薄い(むしろ予測精度が下がる)場合が多いです。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_4.png?raw=true" alt="title" height="170px">

それでは実際に予測モデルを構築してみましょう。

# ゲノミック予測モデルの構築
ここからは実際にゲノミック予測モデルの構築を行っていきます。

今回は、イネの籾数をSNPの遺伝子型から予測するモデルを構築します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_21.png?raw=true" alt="title" height="150px">



## **材料の用意: 遺伝的に多様な集団**

まず初めに、先述した通り、遺伝子型と形質の関係性を捉えるためには、遺伝的に多様な集団のデータを準備する必要があります。

コアコレクション等の多数の系統をベースにしても良いですが、より遺伝子型と形質の関係性を理解するために、**デザインされた育種集団**が材料として良く用いられます。

デザインされた育種集団とは、**RIL集団やNAM集団、MAGIC集団**等を指します。

これらの集団は

* 高い遺伝的多様性
* 固定系統から成るので、雑種強勢やヘテロ接合の影響を無視できる
* 固定系統から成るので、同じ遺伝子型の複製が容易

といった特徴から、予測モデルの構築や育種材料として扱いやすい材料となります。

(雑種強勢なども含めて予測したい場合はF1系統なども混ぜる必要があります。)


本実習では、イネのNAM集団のデータを用いて実習を行います。

NAM(Nested Association Mapping)集団は複数のRecombinant inbred line(RIL)集団から成る集団です.

ある系統を共通親として、共通親と多様な系統からRIL集団を構築したものを総じて、NAM集団と呼びます。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_5.png?raw=true" alt="title" height="300px">

岩手生工研で栽培された、イネ「ひとめぼれ」を共通親として構築したNAM集団のデータを用いて予測モデルを構築していきます。

下のコマンドを動かすとデータがダウンロードされます。



In [ ]:
%%bash
wget -q -O genotype.csv https://raw.githubusercontent.com/slt666666/FAO_lecture/main/FAO_2024/data/genotype.csv
wget -q -O phenotype.csv https://raw.githubusercontent.com/slt666666/FAO_lecture/main/FAO_2024/data/phenotype.csv
wget -q -O modules.py https://raw.githubusercontent.com/slt666666/FAO_lecture/main/FAO_2024/data/modules2.py

ダウンロードを終えると、データの確認が可能です。

形質データは`phenotype.csv`、遺伝子型データは`genotype.csv`になります。

### 形質データ

今回扱うNAM集団は**747系統**あり、各系統を栽培して得られた**籾数(5個体の平均)**のデータがある。

下記のコードを動かすと、Pythonでこのデータを読み込んで表示させることが出来ます。

(現段階で未公開のデータなので、名前はNAM_XXXとなっていますが、複数の系統由来のRIL集団が混ざっています。)

In [ ]:
import pandas as pd
phenotype = pd.read_csv("phenotype.csv")
phenotype

### 遺伝子型データ

今回はNAM集団のSNPの遺伝子型を使用します。

既にこれまでの講義で扱った通り、遺伝子型データを得るためには、シーケンスを行い、アラインメントやSNP callingなどを実施する必要があります。

(SNP検出の流れの一例)

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/Basic_11.png?raw=true" alt="title" height="250px">

今回はNAM集団の各系統毎にシーケンスを行い、SNPの遺伝子型を同定する必要があります。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_6.png?raw=true" alt="title" height="300px">

本実習では、この部分(シーケンス等)はやったものとし、こちらで表データに変換した遺伝子型データを直接使用します。

SNPの遺伝子型は統計モデルで扱いやすいように数値に変換しました。

- 変異がホモ接合型 = 2
- ヘテロ接合型 = 1
- 変異無し(ひとめぼれの遺伝子型) = 0

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_7.png?raw=true" alt="title" height="250px">

下のコードで遺伝子型データをPythonで読み込んで確認することが出来ます。

In [ ]:
import pandas as pd
genotype = pd.read_csv("genotype.csv")
genotype

これで、747系統の遺伝子型データと形質値のデータが手に入りました。

次は実際に、このデータから遺伝子型と形質値の関係を説明できるモデルを構築していきます。


## **予測モデルの構築**

手に入れた遺伝子型と形質値のデータから、その関係性を説明する統計モデルを構築します。

ex) 簡単な例

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_8.png?raw=true" alt="title" height="300px">

近年、計算資源や統計解析手法の発展もあり、様々な統計モデルを簡単に適用することが出来る時代になっています。

そのため、遺伝子間相互作用の考慮や、環境要因の考慮など、目的に応じて適切なモデルを選択し、予測モデルの構築を試みることが出来ます。

(※今回は複雑なモデルは扱いませんが)

<br>

本実習では、シンプルな線形モデルを使用します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_9.png?raw=true" alt="title" height="70px">

殆どの線形モデルはPythonの [scikit-learn](https://scikit-learn.org/stable/) というライブラリ(プログラムがまとめられたもの)を使用することで実装可能です。

```
※ 本実習では時間の都合上、統計については詳しく説明しませんが、
今はオンライン教材が豊富なので必要に応じて学習してみてください。
例) https://pll.harvard.edu/subject/data-science
```

まずは単純な重回帰分析を行ってみます。



In [ ]:
# 線形回帰
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(genotype.iloc[:, 2:].T, phenotype["Grain_number"]) # 遺伝子型情報全体と、籾数の形質値を使用してモデル構築
equation = "Grain number = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

ということで、得られたデータからPythonで重回帰分析をした結果、予測式が出来ました。

## **予測精度の確認**

ただし、すべてのデータを基に予測式を構築したので、このモデルが良いモデルなのかダメなモデルなのか判断できません。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_10.png?raw=true" alt="title" height="120px">

構築したモデルが**他の集団や新しい系統**に対しても上手く予測してくれるのかどうか確認する方法が重要になります。

そこで、通常はモデルの予測精度を簡易的に確認する方法として、データセットを分割して、トレーニングデータとテストデータに分ける方法が用いられます。

### **トレーニングデータ・テストデータへの分割**

データを分割してモデルの精度を検証する方法は、機械学習などのモデル構築の際によく実施されます。

ゲノミック予測モデルを構築する前に、まず得られたデータセットをトレーニングデータとテストデータに分けておきます。

1. まず、トレーニングデータに基づいて予測モデルを構築します。

1. 構築したモデルの精度を確認するために、残りのテストデータを使用します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_11.png?raw=true" alt="title" height="320px">

今回は大体全体の80%のデータがトレーニングデータ、20％のデータがテストデータになるように分割します。

NAM集団が747系統あるので、予測モデルを597系統のデータから構築し、

残りの150系統を使用して構築したモデルの精度を確認する、という形です。

それでは実際にデータを分割して精度を確認してみましょう。

### **1. データの分割**

得られたデータをトレーニングデータとテストデータに分割します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_14.png?raw=true" alt="title" height="300px">

データを分割する際には、形質値の分布が大きく異ならない様に分割します。

(トレーニングデータは小さい個体ばかり、テストデータは大きい個体ばかり、みたいな状況を避ける。)

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_22.png?raw=true" alt="title" height="350px">

[scikit-learn](https://scikit-learn.org/stable/) ライブラリは、形質の分布が概ね一致する様にデータを分割するプログラムも含んでいるので、これを使用します。

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(genotype.iloc[:, 2:].T, phenotype, test_size=0.2, random_state=1024)
plt.figure(figsize=[4,4])
plt.hist(y_train.Grain_number, label="Training data")
plt.hist(y_test.Grain_number, label="Test data")
plt.xlabel("Grain number"); plt.ylabel("Lines"); plt.legend(); plt.show()

In [ ]:
# トレーニングデータの確認
y_train

ということで、形質値の分布が概ね似た形で、トレーニングデータとテストデータに分割できました。

まずはトレーニングデータを用いて予測モデルを構築してみます。

### **2. トレーニングデータから予測モデルを構築**

さきほどやったのと同じように、予測モデルを構築します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_15.png?raw=true" alt="title" height="300px">


[scikit-learn](https://scikit-learn.org/stable/)ライブラリを使用して、単純な重回帰分析でモデルを構築してみます。

In [ ]:
# 線形回帰
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model = model.fit(X_train, y_train["Grain_number"]) # トレーニングデータの遺伝子型と籾数の形質値を使用
equation = "Grain number = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

トレーニングデータの遺伝子型と形質値に基づいた予測モデルが出来ました。


### **3. テストデータを用いて予測精度確認**

では先ほど構築したモデルの精度を残りのデータ(テストデータ)を使用して確認します。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_16.png?raw=true" alt="title" height="300px">

テストデータの遺伝子型から、先ほどのモデルを用いて形質値を予測し、実際の計測した形質値と比較します。

もし予測値と実測値が近ければ、予測が上手くいった、ということになります。

In [ ]:
import numpy as np
y_preds = model.predict(X_test)
plt.scatter(y_test["Grain_number"], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_test["Grain_number"], y_preds)[0, 1]))

上の散布図は実測値(x軸)と予測値(y軸)の関係性を表しています。

実測値が低い(籾数の少ない)系統は、予測値も低くなっていますし、

実測値が高い系統は、予測値も概ね高くなっています。

予測はそこそこ上手くいってそうです。

予測精度を表す指標としては、この実測値と予測値の相関係数(r)を計算することが多いです。

散布図の上に表示してありますが、今回は相関係数が0.81くらいでそれなりの結果だったという感じでしょうか。

遺伝子型と形質値の関係はそれなりに捉えられていると評価できそうです。

これで遺伝子型の情報を基に籾数を予測することが出来る様になりました。

<br>

細かい部分は省略しましたが、この流れがデータを準備して、ゲノミック予測モデルを構築し、精度を確認する流れになります。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_11.png?raw=true" alt="title" height="250px">


## **モデルの中身を見てみる**

さて、今回は単純な重回帰分析を使用したので、すべてのSNPに効果が推定されている形になります。

In [ ]:
equation

数が多すぎて最後まで確認できないので、各SNPの効果をグラフにしてみると…

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(x=range(len(model.coef_)), height=model.coef_)
plt.show()

と、この様な形で、どのSNP(どのゲノム領域)にどのくらいの影響力があるのか確認できます。

ただ見た感じどこの領域もポジティブな効果とネガティブな効果が連続していたりして何だか現実的な感じではない気がします。

そもそも、全ゲノム領域の遺伝子型が形質に影響を与えているとは現実的には考えられず、原因遺伝子が位置する一部の領域の遺伝子型以外に効果があるというのは、おかしい状況です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/GP_not_all_gene.png?raw=true" alt="GP_not_all_gene" height="350px">

この様な事態を考慮して、すべてのSNPではなく、一部の重要なSNPの遺伝子型のみを使用してモデルを構築する方法があります。

使用するSNPを絞って(**変数選択**と呼ばれたりします)モデルを構築する方法としては、大まかに以下の様な方法があります。

* ドメイン知識(生物学の知見等)等から、モデル構築に供与する変数を自分で選択する
* モデル構築する際に変数選択も同時に行ってくれる手法を使う

ドメイン知識(先行研究で機能が重要な遺伝子があればその遺伝子型、既知のQTLやGWASなどで検出された領域)に基づくやり方は、重要なゲノム領域をすべて拾えているか分からないので、あまり良い方法ではないかもしれません。

そこで今回は変数選択を行ってくれる統計手法を試しに扱ってみます。

### (参考)変数選択を伴う手法

先ほどは重回帰分析を行いましたが、今回はElasticNetと呼ばれるモデルを使用してみます。

細かい説明は省きますが、ElasticNetは線形モデルの中でも、使用する変数を絞ってモデルを作っていくことが出来る手法になります。

これも重回帰分析同様、[scikit-learn](https://scikit-learn.org/stable/)ライブラリを使用して、簡単に構築できます。

In [ ]:
# 線形回帰
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=1.0, l1_ratio=0.5) # 使うモデルをElasticNetに
model = model.fit(X_train, y_train["Grain_number"]) # トレーニングデータの遺伝子型と籾数の形質値を使用
equation = "Grain number = "
for i, j in enumerate(model.coef_):
    equation += "SNP{}×{} + ".format(str(i+1), str(j))
equation += str(model.intercept_)
equation

殆どのSNPの効果が0になって、一部のSNPのみ効果を持っている予測式が出来たのが分かるかと思います。

次に、この式の予測精度を見てみると、

In [ ]:
import numpy as np
y_preds = model.predict(X_test)
plt.scatter(y_test["Grain_number"], y_preds)
plt.xlabel("Observed phenotype values")
plt.ylabel("Predicted phenotype values")
print("相関係数: {}".format(np.corrcoef(y_test["Grain_number"], y_preds)[0, 1]))

相関係数が0.84ということで、先ほどより僅かに予測精度が高い予測モデルが出来たことになります。

最後に各SNPの効果を見てみると…

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(x=range(len(model.coef_)), height=model.coef_)
plt.show()

今度は分かりやすい結果になりました、強い効果の領域がどこなのかはっきりと分かります。

当然、与えられたデータに基づいて算出されたSNPの効果なので、これが正しいかどうかは厳密には言えません。

重要なポイントは、これだけの数のSNPで十分にそれなりの予測精度のモデルが構築できる、というポイントです。

恐らく、今回のNAM集団における"籾数"という形質をコントロールしているゲノム領域はそれほど多くない、ということが推察できます。

# ゲノミック予測モデルの活用

それなりに良い精度のゲノミック予測モデルが出来たので、育種にどう活用できるか試してみます。

## **形質を向上させるにはどの様な遺伝子型が良いか**

ゲノミック予測モデルは遺伝子型から形質値を予測できるので、予測モデルに基づいて、形質にとって理想的な遺伝子型を考えることが可能です。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_12.png?raw=true" alt="title" height="120px">

今回は籾数を向上させるような遺伝子型を考えてみましょう。

### 自分で作った遺伝子型の形質値を予測する

自分でよさそうな遺伝子型の系統を考えて、その遺伝子型がどのくらいの形質値になるか予測してみます。

例えば、1番染色体と3番染色体の一部の領域が別の系統と入れ替わった場合、形質値はどうなるか。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_13.png?raw=true" alt="title" height="360px">

この様な情報も、予測モデルに通せば簡単に予測できます。

下のコードを動かすと、計算してくれます。

In [ ]:
from modules import predict_customized_genotype
regions = [['chr01', 10000000, 20000000], ['chr03', 10000000, 20000000]]
predict_customized_genotype(genotype, regions, model)

ひとめぼれ(Cultivar A)が114粒位なのに対し、新しく考えた遺伝子型は120粒と、少し形質が向上するようです。

この様な形で、どんな遺伝子型にすると籾数を向上させることが出来るかが分かるようになりました。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_20.png?raw=true" alt="title" height="360px">

### **Play with genomic prediction**

ということで、色々な遺伝子型を自分で試して、どこまで籾数を伸ばしていくことが出来るかやってみましょう。

下のコードを書き換えて、自分独自の遺伝子型を作成し、籾数を予測してみてください。

2行目の[　]の中身を書き換えたり、書き加えることで、ひとめぼれから別の系統に遺伝子型を変える領域を指定できます。

`regions = [['chr01', 0, 20000000], ['chr03', 10000000, 20000000]]`.

2つ以上の領域を指定する場合は↓の様になります。

`regions = [ ['chrXX', YY, ZZZ], ['chrXX', YY, ZZZ], ['chrXX', YY, ZZZ], ['chrXX', YY, ZZZ] ]`

例えば、↓の様に2行目を変えると、

 `regions = [['chr01', 12000000, 20000000], ['chr05', 0, 5000000], ['chr12', 10000000, 15000000]]`

下の3つの領域が別の系統に変わった場合を計算してくれます。
* chr01...12 Mbp ~ 20 Mbp
* chr05...0 ~ 5 Mbp
* chr12...10 Mbp ~ 15 Mbp

色々試してみてください。

```
よくある書き間違い:
※ Forget  ' ' of 'chr01' and  Forget 0 of 'chr01'
※ Forget comma
※ The number of [ ]
```

※chromosome length is around 20000000bp ~ 40000000bp.


In [ ]:
from modules import predict_customized_genotype
regions = [['chr01', 10000000, 20000000], ['chr03', 10000000, 20000000]]
predict_customized_genotype(genotype, regions, model)

## ゲノミック選抜: 良い次世代を作るために最適な親系統の組み合わせを選抜する

育種系統の遺伝子型情報さえあれば、F2世代などの遺伝子型をシミュレーションすることが出来ます。

シミュレートした遺伝子型から形質値を予測することで、どの組み合わせが良い親系統なのか考えることが出来ます。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/GP_17.png?raw=true" alt="title" height="400px">

今回は例として、NAM集団の中でどの系統を掛け合わせたら良いF2集団が出来るのかを確認してみます。

下のコードは、選んだ2系統からF2集団を作成した場合、どの様な形質の分布を示すか計算してくれるコードです。

"NAM_001" と "NAM_002"の部分を自分の好きな系統(NAM_001~747の範囲で)色々変えて試してみてください。

In [ ]:
from modules import predict_progeny_phenotype
predict_progeny_phenotype("NAM_100", "NAM_002", 100, phenotype, genotype, model)

この様に、良い次世代を作ることが出来る親の組み合わせを試すことが出来ます。

## 参考

今回のモデル活用の例はいずれも「何度も繰り返して良さそうな遺伝子型/親組み合わせを探す」というものでした。

ゲノミック選抜の「育種価が高くなる組み合わせを選抜する」というやり方に近いですね。

しかし、先ほどSNP効果の可視化を行った際、どの領域が大きな効果を持っているかが分かりました。

そこで、その領域だけ別系統の遺伝子型を導入すれば形質が向上することは明白です。

In [ ]:
# 小さな領域で形質を向上させる遺伝子型
from modules import predict_customized_genotype
regions = [['chr07', 28000000, 28300000]]
predict_customized_genotype(genotype, regions, model)

各ゲノム領域の効果を基にすれば、最適な遺伝子も容易に考えられ、最適な親の組み合わせも、ポジティブな効果の領域が出来るだけ多くなる様な組み合わせを考えることでより簡単に高い形質を目指すことが出来ます。

特に、複数の形質を最適化する際には非常に有用な情報となります。

そのため、ゲノミック選抜ではモデルの解釈性というのはブラックボックスでも構わないというのが、従来の考え方でしたが、

特に交配や変異の導入が容易な植物に関しては、モデルの解釈性が高いことは非常に有益だと考えています。

---
## まとめ

このノートブックでは、実際のデータを用いて**ゲノミック予測**を試しました。

ゲノミック予測モデルを用いることで、遺伝子型情報から形質値を予測することができます。

これにより、形質に最適な遺伝子型や親系統の最適な組み合わせを見つけることができます。

遺伝的多様性の高い育種集団(RIL, NAM集団)を作出する計画がある場合、ゲノミック予測に基づく育種アプローチは、高収量・高品質の系統作出を実現するための有用なアプローチの一つだと言えます。
